In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from workflow_widgets import WorkflowWidget
from droplet_workflow import droplet_wf
import workflow_objects as kale

In [3]:
from concurrent.futures import ThreadPoolExecutor

In [4]:
import fireworks as fw
from fireworks.core.rocket_launcher import rapidfire

# Example Workflow

## Define Workflow

In [5]:
t = kale.Task('test')

example_wf = kale.Workflow('example_wf')

n_echos = 10
n_tasks = 4
out_file = "out.txt"

pwd_task = kale.CommandLineTask(
    name='pwd_task',
    command="""echo "Hello from '`pwd`'." > {out_file}""",
    output_files=["{out_file}"],
    params=dict(out_file=out_file)
)

echo_tasks = [
    kale.CommandLineTask(
        name='echo_task_{i}',
        command='for j in {{1..{n_echos}}}; do echo "Hello #{i}: $j" >> {out_file}; sleep 1; done',
        output_files=["{out_file}"],
        params=dict(out_file=out_file, i=i, n_echos=n_echos)
    )
    for i in range(n_tasks)
]

done_task = kale.CommandLineTask(
    name='done_task',
    command="echo 'Done!' >> {out_file}",
    output_files=["{out_file}"],
    params=dict(out_file=out_file)
)

example_wf.add_task(pwd_task)
for i in range(n_tasks):
    example_wf.add_task(echo_tasks[i], dependencies=[pwd_task])
example_wf.add_task(done_task, dependencies=echo_tasks)


In [6]:
cd /home/oliver/lbl/jupyter_hpc

/home/oliver/lbl/jupyter_hpc


In [7]:
pool = kale.WorkerPool(num_workers=4)

2017-08-09 18:53:06,652 INFO Performing db tune-up
2017-08-09 18:53:06,665 INFO LaunchPad was RESET.


In [8]:
pool.log_area.clear_output()
pool.log_area

A Jupyter Widget

In [12]:
pool.fw_run(example_wf)

2017-08-09 18:53:57,614 INFO Added a workflow. id_map: {1: 7, 2: 8, 3: 9, 4: 10, 5: 11, 6: 12}
2017-08-09 18:53:57,643 INFO Created new dir /home/oliver/lbl/jupyter_hpc/launcher_2017-08-10-01-53-57-643424
2017-08-09 18:53:57,664 INFO Created new dir /home/oliver/lbl/jupyter_hpc/launcher_2017-08-10-01-53-57-664249
2017-08-09 18:53:57,695 INFO Launching Rocket
2017-08-09 18:54:08,049 INFO RUNNING fw_id: 9 in directory: /home/oliver/lbl/jupyter_hpc
2017-08-09 18:54:08,053 INFO Task started: ScriptTask.
2017-08-09 18:54:08,075 INFO Task completed: ScriptTask 
2017-08-09 18:54:08,116 INFO Rocket finished


## Visualize

In [10]:
WorkflowWidget(example_wf)

A Jupyter Widget

---

# Droplet Workflow

In [11]:
WorkflowWidget(droplet_wf)

A Jupyter Widget